In [27]:
import cv2
import numpy as np

# โหลดภาพ
image_path = "bloodcell.jpg"
image = cv2.imread(image_path)
if image is None:
    print("ไม่สามารถโหลดภาพได้ กรุณาตรวจสอบที่อยู่ไฟล์")
    exit()

# แปลงเป็นภาพ grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# ปรับความสว่างและความคมชัด
alpha = 1.5  # ปรับความคมชัด
beta = 10    # ปรับความสว่าง
adjusted = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)

# ใช้ CLAHE เพื่อปรับปรุง contrast 
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced = clahe.apply(adjusted)

# ใช้ Gaussian Blur เพื่อลดสัญญาณรบกวน
blurred = cv2.GaussianBlur(enhanced, (1, 1), 0)

# ทำ threshold เพื่อเน้นเฉพาะจุดสีเข้มตรงกลางเซลล์
# ปรับค่า threshold ตามความเข้มของจุดสีเข้มในภาพของคุณ
_, binary = cv2.threshold(blurred, 90, 255, cv2.THRESH_BINARY_INV)

# ใช้ morphological operations เพื่อกำจัดสัญญาณรบกวนเล็กๆ
kernel = np.ones((3,3), np.uint8)
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=1)

# หา contours ของจุดสีเข้ม
contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# กรองเอาเฉพาะ contours ที่มีขนาดเหมาะสม
min_area = 1/20  # ขนาดเล็กสุดของจุดที่ถือว่าเป็นเซลล์
max_area = 150  # ขนาดใหญ่สุดของจุดที่ถือว่าเป็นเซลล์
filtered_contours = []

for contour in contours:
    area = cv2.contourArea(contour)
    if min_area < area < max_area:
        filtered_contours.append(contour)

# หาจุดศูนย์กลางของแต่ละ contour
centroids = []
for contour in filtered_contours:
    M = cv2.moments(contour)
    if M["m00"] > 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        centroids.append((cX, cY))

#วาดวงกลมรอบเซลล์และเพิ่มเลขกำกับ
result = image.copy()
for i, (cX, cY) in enumerate(centroids):
    # วาดวงกลมรอบเซลล์
    cv2.circle(result, (cX, cY), 15, (0, 255, 0), 2)
    # ใส่หมายเลขกำกับ
    cv2.putText(result, str(i+1), (cX-10, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# แสดงจำนวนเซลล์ที่ตรวจพบ
print(f"จำนวนเซลล์ที่ตรวจพบ: {len(centroids)}")

# แสดงภาพและบันทึกผลลัพธ์
cv2.imshow("Original", image)
cv2.imshow("Enhanced", enhanced)
cv2.imshow("Binary", opening)
cv2.imshow("Detected Cells", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

# บันทึกผลลัพธ์
cv2.imwrite("detected_cells_result.jpg", result)

จำนวนเซลล์ที่ตรวจพบ: 129


True